<a href="https://colab.research.google.com/github/inefable12/Exploracion_insilico_2024/blob/main/Consultas_a_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtenido de https://www.youtube.com/watch?v=iDrpdkIHMq8

### 0 - Configuración

In [1]:
!pip install PyPDF2 -q
!pip install langchain -q
!pip install faiss-cpu -q
!pip install openai -q
!pip install python-dotenv -q
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolve

In [2]:
import requests

# Fichero PDF
URL = "https://preguntapdf.s3.eu-south-2.amazonaws.com/BOE-A-1978-31229-consolidado.pdf"
doc_to_download = requests.get(URL)
# Guardar fichero
pdf_file = open("BOE-A-1978-31229-consolidado.pdf", "wb")
pdf_file.write(doc_to_download.content)

646808

### 1 -Leer PDF

In [3]:
from PyPDF2 import PdfReader

In [4]:
# Cargar fichero PDF
pdf_file_obj = open('BOE-A-1978-31229-consolidado.pdf', 'rb')
pdf_reader = PdfReader(pdf_file_obj)

In [5]:
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()

In [6]:
text[8000:10000]

'ESTADO\nLEGISLACIÓN CONSOLIDADA\nPágina 4CAPÍTULO PRIMERO\nDe los españoles y los extranjeros\nArtículo 11.  \n1. La nacionalidad española se adquiere, se conserva y se pierde de acuerdo con lo \nestablecido por la ley.\n2. Ningún español de origen podrá ser privado de su nacionalidad.\n3. El Estado podrá concertar tratados de doble nacionalidad con los países \niberoamericanos o con aquellos que hayan tenido o tengan una particular vinculación con \nEspaña. En estos mismos países, aun cuando no reconozcan a sus ciudadanos un derecho \nrecíproco, podrán naturalizarse los españoles sin perder su nacionalidad de origen.\nArtículo 12.  \nLos españoles son mayores de edad a los dieciocho años.\nArtículo 13.  \n1. Los extranjeros gozarán en España de las libertades públicas que garantiza el \npresente Título en los términos que establezcan los tratados y la ley.\n2. Solamente los españoles serán titulares de los derechos reconocidos en el artículo 23, \nsalvo lo que, atendiendo a criterios

### 2 - Crear chunks

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    length_function=len
    )

In [9]:
chunks = text_splitter.split_text(text)

In [10]:
len(chunks)

176

In [11]:
chunks[13]

'2. Solamente los españoles serán titulares de los derechos reconocidos en el artículo 23, \nsalvo lo que, atendiendo a criterios de reciprocidad, pueda establecerse por tratado o ley \npara el derecho de sufragio activo y pasivo en las elecciones municipales.\n3. La extradición sólo se concederá en cumplimiento de un tratado o de la ley, \natendiendo al principio de reciprocidad. Quedan excluidos de la extradición los delitos \npolíticos, no considerándose como tales los actos de terrorismo.\n4. La ley establecerá los términos en que los ciudadanos de otros países y los apátridas \npodrán gozar del derecho de asilo en España.\nCAPÍTULO SEGUNDO\nDerechos y libertades\nArtículo 14.  \nLos españoles son iguales ante la ley, sin que pueda prevalecer discriminación alguna'

### 3 - Crear Embeddings

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2' # 471M
'sentence-transformers/paraphrase-multilingual-mpnet-base-v2' #1.11G

'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'

In [14]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Ejemplo embedding

In [15]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
sentence_embeddings = model.encode("El perro de san roque no tiene rabo")

In [16]:
len(sentence_embeddings)

384

Crear embeddings de todo el texto

In [17]:
from langchain.vectorstores import FAISS

In [18]:
knowledge_base = FAISS.from_texts(chunks, embeddings)

In [19]:
pregunta = "¿Qué asociaciones quedan expresamente prohibidas por el artículo 22?"
docs = knowledge_base.similarity_search(pregunta, 3)

In [20]:
docs

[Document(page_content='comunicación previa a la autoridad, que sólo podrá prohibirlas cuando existan razones \nfundadas de alteración del orden público, con peligro para personas o bienes.\nArtículo 22.  \n1. Se reconoce el derecho de asociación.\n2. Las asociaciones que persigan fines o utilicen medios tipificados como delito son \nilegales.\n3. Las asociaciones constituidas al amparo de este artículo deberán inscribirse en un \nregistro a los solos efectos de publicidad.\n4. Las asociaciones sólo podrán ser disueltas o suspendidas en sus actividades en virtud \nde resolución judicial motivada.\n5. Se prohíben las asociaciones secretas y las de carácter paramilitar.\nArtículo 23.  \n1. Los ciudadanos tienen el derecho a participar en los asuntos públicos, directamente o'),
 Document(page_content='30.ª Regulación de las condiciones de obtención, expedición y homologación de títulos \nacadémicos y profesionales y normas básicas para el desarrollo del artículo 27 de la \nConstitución, a

### 4 - Preguntar al documento

In [33]:
# dos pociones

# Primera opción: añadir el api key como variable de entorno
#import os
#os.environ["OPENAI_API_KEY"] = "isk-JRY6oNOUHBp31JmeOGikrT3BlbkFJxqQIxSOWLwiYTJEKVUMR"

# Segunda opción: añadir la api key a través de un fichero llamado '.env'
from dotenv import load_dotenv
load_dotenv()

True

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

In [35]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo')

chain = load_qa_chain(llm, chain_type="stuff")

In [36]:
pregunta = "¿Cuántas copas ha ganado el Málaga?"
# Busqueda de párrafos similares
docs = knowledge_base.similarity_search(pregunta, 3)
# Utilizar los parrafos similares para darle contexto a ChatGPT
respuesta = chain.run(input_documents=docs, question=pregunta)
print(f"Respuesta ChatGPT: {respuesta}")

Respuesta ChatGPT: No dispongo de información sobre cuántas copas ha ganado el Málaga.


Review cost

In [37]:
from langchain.callbacks import get_openai_callback

In [38]:
with get_openai_callback() as cb:
    response = chain.run(input_documents=docs, question=pregunta)
    print(cb)

Tokens Used: 762
	Prompt Tokens: 742
	Completion Tokens: 20
Successful Requests: 1
Total Cost (USD): $0.0011530000000000002
